# Import Library

In [1]:
import pandas as pd
import numpy as np
import json

import sys
import os

# Add the directory to the Python path
sys.path.append(os.path.abspath('../'))


from call_function import all_similarity, search_title

c:\Users\Asus\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [45]:
import requests
from bs4 import BeautifulSoup

# Fetch the HTML content from the URL
url = 'https://komikcast.cz/daftar-komik/page/1/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
pagination = soup.find('div', class_='pagination')
page_numbers = pagination.find_all('a', class_='page-numbers')
last_page_number = page_numbers[-2].text

print(last_page_number)

303


<html><body><p>https://komikcast.cz/daftar-komik/page/1</p></body></html>

# Get Data

In [2]:
df_komikcast=pd.read_csv("../data/komikcast.csv")
df_westmanga=pd.read_csv("../data/westmanga.csv")
# df_mangadex=pd.read_csv("./data-processed/mangadex.csv")

print('Komikcast: ', df_komikcast.shape)
print('Westmanga: ', df_westmanga.shape)
# print('Mangadex : ', df_mangadex.shape)

Komikcast:  (8410, 12)
Westmanga:  (4810, 12)


In [5]:
title = 'fairy tail'
data = search_title(title, df_komikcast)
datas = df_komikcast[df_komikcast['title'] == data]
datas

,id,title,alt_title,type,description,genre,author,artist,rate,image,released,image_hash
373,374,Fairy Tail,FAIRY TAIL,Manga,Dalam dunia mistik Tanah Bumi magic ada pada i...,"Action, Adventure, Comedy, Ecchi, Fantasy, Mag...","Mashima, Hiro (Story & Art)",-,8.22,https://komikcast.cz/wp-content/uploads/2017/0...,23-Aug-06,92d8a4f0844cdcff


In [16]:
# Load the JSON file
with open("../data/consine_decription_similarity.json", "r") as infile:
    consine_decription_similarity = json.load(infile)

In [17]:
# Inverse: Convert lists back to NumPy arrays
consine_decription_similarity = {
    key: np.array(value) for key, value in consine_decription_similarity.items()
}

consine_sim_indo_komikcast = consine_decription_similarity['komikcast']
consine_sim_indo_westmanga = consine_decription_similarity['westmanga']

In [22]:
title = 'Fairy Tail'

In [26]:
df_komikcast_similarity = all_similarity(title, df_komikcast,consine_sim_indo_komikcast)
df_komikcast_similarity.head(2)

Exact title not found. Using closest match: 'Fairy Tail City Hero'


,id,image_similarity,description_similarity,title,alt_title,type,description,genre,author,artist,rate,image,released,image_hash,genre_similarity
5842,679,0.406250,0.025302,Shitsugyou Kenja no Nariagari,"The Rise of the Unemployed Wise Man, 失業賢者の成り上か...",Manga,Karna adalah anak lakilaki yang diberi gelar S...,"Action, Adventure, Comedy, Ecchi, Fantasy, Mag...",MIKAWA Ghost,-,7.0,https://komikcast.cz/wp-content/uploads/2021/1...,2020,ceacf8f8e0010bff,87.5
7503,7558,0.296875,0.002305,12 Beast,12BEAST,Manga,Bercerita tentang Eita Touga seorang ninja mag...,"Action, Adventure, Comedy, Drama, Ecchi, Fanta...","Inui, Takemaru (Story & Art)",-,7.1,https://komikcast.cz/wp-content/uploads/2017/0...,9-May-13,fffff18498d0f0f6,87.5


In [28]:
df_westmanga_similarity = all_similarity(title, df_westmanga,consine_sim_indo_westmanga)
df_westmanga_similarity.tail(2)

Exact title not found. Using closest match: 'Fairy Tail City Hero'
Exact title not found. Using closest match: 'Fairy Tail City Hero'
Exact title not found. Using closest match: 'Fairy Tail 100 Years Quest'
Error processing image hash for title 'King of Runes': invalid literal for int() with base 16: '7.08E+19'


,id,image_similarity,description_similarity,title,alt_title,type,description,genre,author,artist,rate,image,released,image_hash,genre_similarity
4089,3867,0.40625,0.014652,Tamamo no Koi,"玉藻の恋, 玉藻狐之恋",Manga,Setelah aku mengunjungi Kuil Shinto di luar ko...,"Romance, Seinen, Slice of Life, Supernatural",Mito (ミト),-,7.3,https://westmanga.fun/wp-content/uploads/2021/...,-,46fff6e660404f06,0.0
1885,442,0.53125,0.000000,Boku to Kimi no Taisetsu na Hanashi,"Our Precious Conversations, 僕と君の大切な話",Manga,Sebuah manga komedi romantis anak SMA yang man...,"Romance, School, Shoujo",Robico (Story & Art),-,6.0,https://westmanga.fun/wp-content/uploads/2021/...,-,0099fbcbcbcbfbf1,0.0


In [25]:
# df_mangadex  = all_similarity(title, df_mangadex ,consine_sim_eng_mangadex)
# df_mangadex.head()

In [32]:
# Import Library
import pandas as pd
import numpy as np
import math
import requests
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import re
from flask import Flask, request, jsonify

def get_comics_mangadex(url):
    url_stat = 'https://api.mangadex.org/statistics/manga?manga[]='
    response = requests.get(url)
    if response.status_code == 200:
        json = []
        lan = ['en', 'ja', 'ko', 'ru', 'zh', 'ko-ro', 'zh-hk','es-la']
        data = response.json()

        for comic in data['data']:
            url_manga = 'https://api.mangadex.org/manga'
            url_cover = 'https://mangadex.org/covers'
            id = comic['id']
            
            title = next((v for k, v in comic['attributes']['title'].items() if k in lan), None)
            description = next((v for k, v in comic['attributes']['description'].items() if k in lan), None)
            clean_title = re.sub(r'[^a-zA-Z0-9\s]', '', title)
            formatted_title = clean_title.replace(' ', '-')
            comic_url = f'https://mangadex.org/title/{id}/{formatted_title}'
            
            alt_titles = []
            for alt_title_dict in comic['attributes'].get('altTitles', []):
                alt_title = next((v for k, v in alt_title_dict.items() if k in lan), None)
                if alt_title:
                    alt_titles.append(alt_title)
            alt_titles = ', '.join(alt_titles)
            
            genres = []
            for tag in comic['attributes'].get('tags', []):
                tag_name = next((v for k, v in tag['attributes']['name'].items() if k in ['en', 'ja', 'ko']), None)
                if tag_name:
                    genres.append(tag_name)
            genres = ', '.join(genres)
        
            released = comic['attributes']['year'] if comic['attributes']['year'] and int(comic['attributes']['year']) else '-'
            
            rate = 0
            response_rate = requests.get(url_stat+id)
            if response_rate.status_code == 200:
                data_stat = response_rate.json()
                rate = data_stat['statistics'][id]['rating']['average']

            author = ''
            artist = ''
            img = ''
            url_manga += '/'+id+'?includes[]=artist&includes[]=author&includes[]=cover_art'
            response_manga = requests.get(url_manga)
            if response_manga.status_code == 200:
                data_manga = response_manga.json()
                for relation in data_manga['data']['relationships']:
                    if relation['type'] == 'author' and 'attributes' in relation and 'name' in relation['attributes']:
                        author = relation['attributes']['name']
                    if relation['type'] == 'artist' and 'attributes' in relation and 'name' in relation['attributes']:
                        artist = relation['attributes']['name']
                    if relation['type'] == 'cover_art' and 'attributes' in relation and 'fileName' in relation['attributes']:
                        img = f"{url_cover}/{id}/{relation['attributes']['fileName']}"

            json.append({
                'title': title,
                'alt_title': alt_titles,
                'type': comic['type'],
                'description': description,
                'genre': genres,
                'author':author,
                'artist':artist,
                'rate': rate,
                'image': img,
                'released': released,
                'comic_url': comic_url,
            })
            
        return json
    
url = f'https://api.mangadex.org/manga?limit=32&offset=32'
result = get_comics_mangadex(url)
result[0]

{'title': 'Yuusha Party ni Kawaii Ko ga Ita no de, Kokuhaku Shite Mita',
 'alt_title': "勇者パーティーにかわいい子がいたので、告白してみた。, There Was a Cute Girl in the Hero's Party, so I Tried Confessing to Her, 身为魔族的我想向勇者小队的可爱女孩告白, 용사 파티에 귀여운 애가 있어서, 고백해봤다",
 'type': 'manga',
 'description': "Youki was reincarnated in another world. However, instead of being reincarnated as the hero or the demon lord, he was instead reincarnated half-assedly as a minion of the Demon Lord. One day, the Hero's Party attacked the Demon Lord's castle with the intent of defeating the Demon Lord. Of course, Youki was able to repel their attempt with ease. In spite of this, he ended up falling for the cleric, Cecilia, at first sight… The super popular love comedy fantasy set in another world!",
 'genre': 'Reincarnation, Demons, Romance, Comedy, Adventure, Magic, Isekai, Fantasy, Adaptation',
 'author': 'Suisei',
 'artist': 'Kairi',
 'rate': 8.2657,
 'image': 'https://mangadex.org/covers/024af7a5-f8ad-4b67-b950-b1708f52b018/180d709